# What's new in the Forecastwrapper

• Solar Irradiance!

# Demo of the forecast.io wrapper to get past and future weather data

Important: you need to register for an apikey here: https://developer.forecast.io/  Put the key you obtain in the opengrid.cfg file as follows:

    [Forecast.io]
    apikey: your_key

In [ ]:
import os
import sys
import inspect
import pandas as pd
import charts

## Import API wrapper module

In [ ]:
from opengrid.library import forecastwrapper

# Get weather data in daily and hourly resolution

To get started, create a Weather object for a certain location and a period

In [ ]:
start = pd.Timestamp('20150813')
end = pd.Timestamp('20150816')

In [ ]:
Weather_Ukkel = forecastwrapper.Weather(location='Ukkel', start=start, end=end)

You can use the methods `days()` and `hours()` to get a dataframe in daily or hourly resolution

In [ ]:
Weather_Ukkel.days().info()

In [ ]:
Weather_Ukkel.hours().info()

### Degree Days

Daily resolution has the option of adding degree days.
By default, the temperature equivalent and heating degree days with a base temperature of 16.5°C are added.

Heating degree days are calculated as follows:

$$heatingDegreeDays = max(0 , baseTemp - 0.6 * T_{today} + 0.3 * T_{today-1} + 0.1 * T_{today-2} )$$

Cooling degree days are calculated in an analog way:

$$coolingDegreeDays = max(0, 0.6 * T_{today} + 0.3 * T_{today-1} + 0.1 * T_{today-2} - baseTemp )$$

Add degree days by setting the flag `include_heating_degree_days` or `include_cooling_degree_days` and supplying `heating_base_temperatures` and/or `cooling_base_temperatures` as a list (you can add multiple base temperatures, or just a list of 1 element)

#### Get some more degree days

In [ ]:
Weather_Ukkel.days(include_heating_degree_days = True,
                   heating_base_temperatures = [15,18],
                   include_cooling_degree_days = True,
                   cooling_base_temperatures = [18,24]).filter(like='DegreeDays')

In [ ]:
Weather_Ukkel.days()

### Daytime Averages

Daily resolution has the option of adding average values during the daytime only (between sunrise and sunset). Implemented are daytimeCloudCover and daytimeTemperature, and are included by default. They can be excluded by setting `include_daytime_cloud_cover = False` or `include_daytime_temperature = False`.

By default, daily weather data is extended with *daytimeCloudCover* and *daytimeTemperature*.  These are the averaged cloud cover and temperature **between sunrise and sunset**. 

In [ ]:
Weather_Ukkel.days().filter(like='daytime')

# Hourly resolution example

Location can also be coördinates

In [ ]:
start = pd.Timestamp('20150916')
end = pd.Timestamp('20150918')
Weather_Brussel = forecastwrapper.Weather(location=[50.8503396, 4.3517103], start=start, end=end)
Weather_Boutersem = forecastwrapper.Weather(location='Kapelstraat 1, 3370 Boutersem', start=start, end=end)

In [ ]:
df_combined = pd.merge(Weather_Brussel.hours(), Weather_Boutersem.hours(), suffixes=('_Brussel', '_Boutersem'), 
                       left_index=True, right_index=True)

In [ ]:
charts.plot(df_combined.filter(like='cloud'), stock=True, show='inline')

# Deal with different timezones

By default, all dataframes are localized to the timezone of the specified location.
If you want, you can specify the tz argument to get all dataframes localized to a specified timezone.

In [ ]:
start = pd.Timestamp('20150815')
end = pd.Timestamp('20150816')

In [ ]:
Weather_London = forecastwrapper.Weather(location='London', start=start, end=end, tz='Asia/Singapore')
Weather_Brussels = forecastwrapper.Weather(location='Brussels', start=start, end=end, tz='Asia/Singapore')

In [ ]:
Weather_London.days()

In [ ]:
Weather_Brussels.days()

## Built-In Caching

Caching is turned on by default, so when you try and get dataframes the first time it takes a long time...

In [ ]:
start = pd.Timestamp('20161020')
end = pd.Timestamp('20161030')

In [ ]:
Weather_Ukkel = forecastwrapper.Weather(location='Ukkel', start=start, end=end)
Weather_Ukkel.days().head(1)

... but now try that again and it goes very fast

In [ ]:
Weather_Ukkel = forecastwrapper.Weather(location='Ukkel', start=start, end=end)
Weather_Ukkel.days().head(1)

You can turn of the behaviour by setting the cache flag to false:

In [ ]:
Weather_Ukkel = forecastwrapper.Weather(location='Ukkel', start=start, end=end, cache=False)

## Solar Irradiance!

Dark Sky has added Solar Irradiance data as a beta.

Note:
- The values are calculated, not measured. Dark Sky uses the position of the sun in combination with cloud cover.
- Western Europe is not in Dark Sky's "primary region", therefore the data is not super-accurate.
- Since it is a beta, the algorithms and therefore the values may change
- I (JrtPec) have done a qualitative analysis that compared these values with those measured by KNMI (Netherlands). The differences were significant (27% lower). I have notified Dark Sky and they will investigate and possibly update their algorithms.
- You need to delete your cached files in order to include these new values (everything will have to be re-downloaded)
- If Dark Sky were to update their values, the cache needs to be deleted again.

### Hourly data

In [ ]:
Weather_Ukkel.hours()[[
        'GlobalHorizontalIrradiance',
        'DirectNormalIrradiance',
        'DiffuseHorizontalIrradiance',
        'ExtraTerrestrialRadiation',
        'SolarAltitude',
        'SolarAzimuth']].dropna().head()

- Global Horizontal Irradiance is the amount of Solar Irradiance that shines on a horizontal surface, direct and diffuse, in Wh/m<sup>2</sup>. It is calculated by transforming the Direct Normal Irradiance (DNI) to the horizontal plane and adding the Diffuse Horizontal Irradiance (DHI):

$$GHI = DNI * cos(90° - Altitude) + DHI$$

- The GHI is what you would use to benchmark PV-panels

- Direct Normal Irradiance is the amount of solar irradiance that shines directly on a plane tilted towards the sun. In Wh/m<sup>2</sup>.

- Diffuse Horizontal Irradiance is the amount of solar irradiance that is scattered in the atmosphere and by clouds. In Wh/m<sup>2</sup>.

- Extra-Terrestrial Radiation is the GHI a point would receive if there was no atmosphere.

- Altitude of the Sun is measured in degrees above the horizon.

- Azimuth is the direction of the Sun in degrees, measured from the true north going clockwise.

At night, all values will be `NaN`

### Daily data

The daily sum of the GHI is included in the `day` dataframe. Values are in Wh/m<sup>2</sup>

If you need other daily aggregates, give me a shout!

In [ ]:
Weather_Ukkel.days().GlobalHorizontalIrradiance.head()